In [1]:
import csv
import random
import json
import numpy as np
from datetime import datetime, timedelta

# Список активностей
activities = [
    "клик по рекламе",
    "клик на переход с информации на ввод телефона",
    "ввод телефона",
    "ввод смс",
    "ввод паспорта",
    "ввод почты",
    "ввод фио",
    "сабмит",
    "приход в отделение",
    "взял займ"
    ]

# Генерация случайного значения номера телефона
def generate_phone_number():
    phone_number = "".join(random.choices("0123456789", k = 10))
    return phone_number

# Генерация случайного значения кода из смс
def generate_sms_code():
    sms_code = "".join(random.choices("0123456789", k = 4))
    return sms_code

# Генерация случайного значения номера паспорта
def generate_passport_number():
    passport_number = "".join(random.choices("0123456789", k = 10))
    return passport_number

# Генерация случайного значения электронной почты
def generate_email():
    email = f"user{random.randint(1, 10000)}@example.com"
    return email

# Генерация случайного значения фамилии, имени и отчества
def generate_full_name():
    first_names = ["John", "Emma", "Michael", "Sophia", "William", "Olivia", "James", "Ava", "Benjamin", "Isabella",
                   "Daniel", "Mia", "Elijah", "Charlotte", "Alexander", "Amelia", "Henry", "Harper", "Matthew", "Evelyn"]

    last_names = ["Smith", "Johnson", "Williams", "Jones", "Brown", "Davis", "Miller", "Wilson", "Moore", "Taylor",
                  "Anderson", "Thomas", "Jackson", "White", "Harris", "Martin", "Thompson", "Garcia", "Martinez", "Clark"]

    middle_names = ["Robert", "David", "Joseph", "Charles", "Thomas", "Daniel", "Matthew", "Elizabeth", "Marie", "Jane",
                    "Alice", "Emily", "Liam", "David", "Oliver", "Noah", "Ethan", "Lucas", "Mason", "Logan"]

    full_name = {
        "last_name": random.choice(last_names),
        "first_name": random.choice(first_names),
        "middle_name": random.choice(middle_names)
    }
    return full_name

# Генерация случайного значения запрошеных клиентом денег
def generate_money():
    x = random.random()
    if x <= 0.4:
        return random.randint(10000, 50000)
    elif x <= 0.7:
        return random.randint(50000, 100000)
    else:
        return random.randint(100000, 200000)



# Генерация вероятности взять кредит от суммы кредита
def probability_of_taking(x):
    if x <= 50000:
        return 0.95
    elif x <= 100000:
        return 0.3 + 0.3 * np.log10(x - 50000) / np.log10(100000 - 50000)
    elif x <= 200000:
        return 0.05 + 0.35 * np.log10(x - 100000) / np.log10(200000 - 100000)


# Генерация фичей в зависимости от вида активности
def generate_features(activity, t_p_10 = 70, t_p_2 = 1, t_p_0 = 2, t_p_b = 1,
                       t_i_10 = 75, t_i_2 = 2, t_i_0 = 0, t_i_b = 2,
                       t_e_10 = 65, t_e_2 = 1, t_e_0 = 2, t_e_b = 4,
                       t_f_10 = 80, t_f_2 = 0, t_f_0 = 1, t_f_b = 2):
    if activity == "ввод телефона":
        value = generate_phone_number()
        num_of_keypress = random.choices([10] * t_p_10 + [2] * t_p_2 + [0] * t_p_0 + list(range(12, 27, 2))* t_p_b, k=1)[0]
        num_of_backspace = 0 if num_of_keypress <= 10 else (num_of_keypress - 10) // 2
        features = {
            "value": value,
            "num_of_keypress": num_of_keypress,
            "num_of_backspace": num_of_backspace
        }
    elif activity == "ввод смс":
        value = generate_sms_code()
        num_of_keypress = random.choices([4] * 60 + [0] * 20 + list(range(6, 9, 2)), k=1)[0]
        num_of_backspace = 0 if num_of_keypress <= 4 else (num_of_keypress - 4) // 2
        features = {
            "value": value,
            "num_of_keypress": num_of_keypress,
            "num_of_backspace": num_of_backspace
        }
    elif activity == "ввод паспорта":
        value = generate_passport_number()
        num_of_keypress = random.choices([10] * t_i_10 + [2] * t_i_2 + [0] * t_i_0 + list(range(10, 21, 2)) * t_i_b, k=1)[0]
        num_of_backspace = 0 if num_of_keypress <= 10 else (num_of_keypress - 10) // 2
        features = {
            "value": value,
            "num_of_keypress": num_of_keypress,
            "num_of_backspace": num_of_backspace
        }
    elif activity == "ввод почты":
        value = generate_email()
        email_length = len(value)
        num_of_keypress = random.choices([email_length] * t_e_10 + [2] * t_e_2 + [0] * t_e_0 + list(range(email_length, email_length + 11, 2)) * t_e_b, k=1)[0]
        num_of_backspace = 0 if num_of_keypress <= email_length else (num_of_keypress - email_length) // 2
        features = {
            "value": value,
            "num_of_keypress": num_of_keypress,
            "num_of_backspace": num_of_backspace
        }
    elif activity == "ввод фио":
        full_name = generate_full_name()
        value = f"{full_name['last_name']} {full_name['first_name']} {full_name['middle_name']}"
        full_name_length = len(full_name)
        num_of_keypress = random.choices([full_name_length] * t_f_10 + [2] * t_f_2 + [0] * t_f_0 + list(range(full_name_length, full_name_length + 11, 2)) * t_f_b, k=1)[0]
        num_of_backspace = 0 if num_of_keypress <= full_name_length else (num_of_keypress - full_name_length) // 2

        features = {
            "value": value,
            "num_of_keypress": num_of_keypress,
            "num_of_backspace": num_of_backspace
        }
    elif activity == "сабмит":
        value = generate_money()
        features = {
            "value": value
        }

    else:
        features = None

    return features

# Генерация временной метки
def generate_timestamp(start_timestamp, seconds):
    timestamp = start_timestamp + timedelta(seconds=seconds)
    return timestamp.strftime("%Y-%m-%d %H:%M:%S")

# Генерация данных для csv
def generate_data():
    data = []
    client_ids = list(range(1, 100001))
    random.shuffle(client_ids)

    # 10% клиентов по порядку делают все действия
    for client_id in client_ids[:7500]:
        start_timestamp = datetime.now() - timedelta(days=random.randint(0, 500))
        seconds = 0
        for activity in activities[:9]:
            timestamp = generate_timestamp(start_timestamp, seconds)
            if activity  == 'клик по рекламе':
              seconds += random.randint(500, 800)
            elif activity  == 'клик на переход с информации на ввод телефона':
              seconds += random.randint(1, 15)
            elif activity  == 'ввод телефона':
              seconds += random.randint(50, 70)
            elif activity  == 'ввод смс':
              seconds += random.randint(40, 70)
            elif activity  == 'ввод паспорта':
              seconds += random.randint(180, 400)
            elif activity  == 'ввод почты':
              seconds += random.randint(15, 60)
            elif activity  == 'ввод фио':
              seconds += random.randint(1, 60)
            elif activity  == 'сабмит':
              seconds += random.randint(80000, 216000)
            else:
              seconds += random.randint(720, 7200)
            features = generate_features(activity, 80, 0, 0, 6, 80, 0, 0, 8, 70, 1, 0, 10, 85, 0, 1, 7)
            row = {
                "timestamp": timestamp,
                "client_id": client_id,
                "activity": activity,
                "features": json.dumps(features)
            }
            if activity == "сабмит":
                money = features['value']
            data.append(row)
        if probability_of_taking(money) > random.random():
            timestamp = generate_timestamp(start_timestamp, seconds)
            features = generate_features(activities[9])
            row = {
            "timestamp": timestamp,
            "client_id": client_id,
            "activity": activities[9],
            "features": None
            }
            data.append(row)

    # 30% клиентов делают только первую активность
    for client_id in client_ids[7500:38000]:
        start_timestamp = datetime.now() - timedelta(days=random.randint(0, 500))
        timestamp = generate_timestamp(start_timestamp, 0)
        row = {
            "timestamp": timestamp,
            "client_id": client_id,
            "activity": activities[0],
            "features": None
        }
        data.append(row)

    # 20% клиентов делают первые две активности
    for client_id in client_ids[38000:61000]:
        start_timestamp = datetime.now() - timedelta(days=random.randint(0, 500))
        seconds = 0
        for activity in activities[:2]:
            timestamp = generate_timestamp(start_timestamp, seconds)
            if activity  == 'клик по рекламе':
              seconds += random.randint(150, 500)
            elif activity  == 'клик на переход с информации на ввод телефона':
              seconds += random.randint(1, 13)
            elif activity  == 'ввод телефона':
              seconds += random.randint(30, 60)
            elif activity  == 'ввод смс':
              seconds += random.randint(20, 60)
            elif activity  == 'ввод паспорта':
              seconds += random.randint(15, 300)
            elif activity  == 'ввод почты':
              seconds += random.randint(5, 45)
            elif activity  == 'ввод фио':
              seconds += random.randint(1, 50)
            elif activity  == 'сабмит':
              seconds += random.randint(230000, 430000)
            else:
              seconds += random.randint(720, 7200)

            features = generate_features(activity)
            row = {
                "timestamp": timestamp,
                "client_id": client_id,
                "activity": activity,
                "features": json.dumps(features)
            }
            data.append(row)

    # 5% клиентов делают первые три активности
    for client_id in client_ids[61000:64000]:
        start_timestamp = datetime.now() - timedelta(days=random.randint(0, 500))
        seconds = 0
        previous_activity = None
        for activity in activities[:3]:
            timestamp = generate_timestamp(start_timestamp, seconds)
            if activity  == 'клик по рекламе':
              seconds += random.randint(150, 500)
            elif activity  == 'клик на переход с информации на ввод телефона':
              seconds += random.randint(1, 13)
            elif activity  == 'ввод телефона':
              seconds += random.randint(30, 60)
            elif activity  == 'ввод смс':
              seconds += random.randint(20, 60)
            elif activity  == 'ввод паспорта':
              seconds += random.randint(15, 300)
            elif activity  == 'ввод почты':
              seconds += random.randint(5, 45)
            elif activity  == 'ввод фио':
              seconds += random.randint(1, 50)
            elif activity  == 'сабмит':
              seconds += random.randint(230000, 430000)
            else:
              seconds += random.randint(720, 7200)
            features = generate_features(activity)
            row = {
                "timestamp": timestamp,
                "client_id": client_id,
                "activity": activity,
                "features": json.dumps(features)
            }

            if previous_activity is not None and previous_activity != "клик по рекламе" and previous_activity != "ввод фио" and previous_activity != "сабмит" and previous_activity != "приход в отделение" and random.random() < 0.2:
                data.append({
                    "timestamp": generate_timestamp(start_timestamp, seconds),
                    "client_id": client_id,
                    "activity": previous_activity,
                    "features": json.dumps(generate_features(previous_activity))
                })
                seconds += random.randint(1, 500)
                data.append({
                    "timestamp": generate_timestamp(start_timestamp, seconds),
                    "client_id": client_id,
                    "activity": activity,
                    "features": json.dumps(features)
                })
                seconds += random.randint(1, 500)
            data.append(row)
            previous_activity = activity

    # 5% клиентов делают первые четыре активности
    for client_id in client_ids[64000:67000]:
        start_timestamp = datetime.now() - timedelta(days=random.randint(0, 500))
        seconds = 0
        previous_activity = None
        for activity in activities[:4]:
            timestamp = generate_timestamp(start_timestamp, seconds)
            if activity  == 'клик по рекламе':
              seconds += random.randint(150, 500)
            elif activity  == 'клик на переход с информации на ввод телефона':
              seconds += random.randint(1, 13)
            elif activity  == 'ввод телефона':
              seconds += random.randint(30, 60)
            elif activity  == 'ввод смс':
              seconds += random.randint(20, 60)
            elif activity  == 'ввод паспорта':
              seconds += random.randint(15, 300)
            elif activity  == 'ввод почты':
              seconds += random.randint(5, 45)
            elif activity  == 'ввод фио':
              seconds += random.randint(1, 50)
            elif activity  == 'сабмит':
              seconds += random.randint(230000, 430000)
            else:
              seconds += random.randint(720, 7200)
            features = generate_features(activity)
            row = {
                "timestamp": timestamp,
                "client_id": client_id,
                "activity": activity,
                "features": json.dumps(features)
            }

            if previous_activity is not None and previous_activity != "клик по рекламе"  and previous_activity != "ввод фио" and previous_activity != "сабмит" and previous_activity != "приход в отделение" and random.random() < 0.25:
                data.append({
                    "timestamp": generate_timestamp(start_timestamp, seconds),
                    "client_id": client_id,
                    "activity": previous_activity,
                    "features": json.dumps(generate_features(previous_activity))
                })
                seconds += random.randint(1, 500)
                data.append({
                    "timestamp": generate_timestamp(start_timestamp, seconds),
                    "client_id": client_id,
                    "activity": activity,
                    "features": json.dumps(features)
                })
                seconds += random.randint(1, 500)
            data.append(row)
            previous_activity = activity

    # 5% клиентов делают первые пять активностей
    for client_id in client_ids[67000:70000]:
        start_timestamp = datetime.now() - timedelta(days=random.randint(0, 500))
        seconds = 0
        previous_activity = None
        for activity in activities[:5]:
            timestamp = generate_timestamp(start_timestamp, seconds)
            if activity  == 'клик по рекламе':
              seconds += random.randint(150, 500)
            elif activity  == 'клик на переход с информации на ввод телефона':
              seconds += random.randint(1, 13)
            elif activity  == 'ввод телефона':
              seconds += random.randint(30, 60)
            elif activity  == 'ввод смс':
              seconds += random.randint(20, 60)
            elif activity  == 'ввод паспорта':
              seconds += random.randint(15, 300)
            elif activity  == 'ввод почты':
              seconds += random.randint(5, 45)
            elif activity  == 'ввод фио':
              seconds += random.randint(1, 50)
            elif activity  == 'сабмит':
              seconds += random.randint(230000, 430000)
            else:
              seconds += random.randint(720, 7200)
            features = generate_features(activity)
            row = {
                "timestamp": timestamp,
                "client_id": client_id,
                "activity": activity,
                "features": json.dumps(features)
            }

            if previous_activity is not None and previous_activity != "клик по рекламе"  and previous_activity != "ввод фио" and previous_activity != "сабмит" and previous_activity != "приход в отделение" and random.random() < 0.15:
                data.append({
                    "timestamp": generate_timestamp(start_timestamp, seconds),
                    "client_id": client_id,
                    "activity": previous_activity,
                    "features": json.dumps(generate_features(previous_activity))
                })
                seconds += random.randint(1, 500)
                data.append({
                    "timestamp": generate_timestamp(start_timestamp, seconds),
                    "client_id": client_id,
                    "activity": activity,
                    "features": json.dumps(features)
                })
                seconds += random.randint(1, 500)
            data.append(row)
            previous_activity = activity

    # 5% клиентов делают первые шесть активностей
    for client_id in client_ids[70000:73000]:
        start_timestamp = datetime.now() - timedelta(days=random.randint(0, 500))
        seconds = 0
        previous_activity = None
        for activity in activities[:6]:
            timestamp = generate_timestamp(start_timestamp, seconds)
            if activity  == 'клик по рекламе':
              seconds += random.randint(150, 500)
            elif activity  == 'клик на переход с информации на ввод телефона':
              seconds += random.randint(1, 13)
            elif activity  == 'ввод телефона':
              seconds += random.randint(30, 60)
            elif activity  == 'ввод смс':
              seconds += random.randint(20, 60)
            elif activity  == 'ввод паспорта':
              seconds += random.randint(15, 300)
            elif activity  == 'ввод почты':
              seconds += random.randint(5, 45)
            elif activity  == 'ввод фио':
              seconds += random.randint(1, 50)
            elif activity  == 'сабмит':
              seconds += random.randint(230000, 430000)
            else:
              seconds += random.randint(720, 7200)
            features = generate_features(activity)
            row = {
                "timestamp": timestamp,
                "client_id": client_id,
                "activity": activity,
                "features": json.dumps(features)
            }

            if previous_activity is not None and previous_activity != "клик по рекламе"  and previous_activity != "ввод фио" and previous_activity != "сабмит" and previous_activity != "приход в отделение" and random.random() < 0.15:
                data.append({
                    "timestamp": generate_timestamp(start_timestamp, seconds),
                    "client_id": client_id,
                    "activity": previous_activity,
                    "features": json.dumps(generate_features(previous_activity))
                })
                seconds += random.randint(1, 500)
                data.append({
                    "timestamp": generate_timestamp(start_timestamp, seconds),
                    "client_id": client_id,
                    "activity": activity,
                    "features": json.dumps(features)
                })
                seconds += random.randint(1, 500)
            data.append(row)
            previous_activity = activity

    # 5% клиентов делают первые семь активностей
    for client_id in client_ids[73000:76000]:
        start_timestamp = datetime.now() - timedelta(days=random.randint(0, 500))
        seconds = 0
        previous_activity = None
        for activity in activities[:7]:
            timestamp = generate_timestamp(start_timestamp, seconds)
            if activity  == 'клик по рекламе':
              seconds += random.randint(150, 500)
            elif activity  == 'клик на переход с информации на ввод телефона':
              seconds += random.randint(1, 13)
            elif activity  == 'ввод телефона':
              seconds += random.randint(30, 60)
            elif activity  == 'ввод смс':
              seconds += random.randint(20, 60)
            elif activity  == 'ввод паспорта':
              seconds += random.randint(15, 300)
            elif activity  == 'ввод почты':
              seconds += random.randint(5, 45)
            elif activity  == 'ввод фио':
              seconds += random.randint(1, 50)
            elif activity  == 'сабмит':
              seconds += random.randint(230000, 430000)
            else:
              seconds += random.randint(720, 7200)
            features = generate_features(activity)
            row = {
                "timestamp": timestamp,
                "client_id": client_id,
                "activity": activity,
                "features": json.dumps(features)
            }

            if previous_activity is not None and previous_activity != "клик по рекламе"  and previous_activity != "ввод фио" and previous_activity != "сабмит" and previous_activity != "приход в отделение" and random.random() < 0.1:
                data.append({
                    "timestamp": generate_timestamp(start_timestamp, seconds),
                    "client_id": client_id,
                    "activity": previous_activity,
                    "features": json.dumps(generate_features(previous_activity))
                })
                seconds += random.randint(1, 500)
                data.append({
                    "timestamp": generate_timestamp(start_timestamp, seconds),
                    "client_id": client_id,
                    "activity": activity,
                    "features": json.dumps(features)
                })
                seconds += random.randint(1, 500)
            data.append(row)
            previous_activity = activity

    # 5% клиентов делают первые восемь активностей
    for client_id in client_ids[76000:90000]:
        start_timestamp = datetime.now() - timedelta(days=random.randint(0, 500))
        seconds = 0
        previous_activity = None
        for activity in activities[:8]:
            timestamp = generate_timestamp(start_timestamp, seconds)
            if activity  == 'клик по рекламе':
              seconds += random.randint(150, 500)
            elif activity  == 'клик на переход с информации на ввод телефона':
              seconds += random.randint(1, 13)
            elif activity  == 'ввод телефона':
              seconds += random.randint(30, 60)
            elif activity  == 'ввод смс':
              seconds += random.randint(20, 60)
            elif activity  == 'ввод паспорта':
              seconds += random.randint(15, 300)
            elif activity  == 'ввод почты':
              seconds += random.randint(5, 45)
            elif activity  == 'ввод фио':
              seconds += random.randint(1, 50)
            elif activity  == 'сабмит':
              seconds += random.randint(230000, 430000)
            else:
              seconds += random.randint(720, 7200)
            features = generate_features(activity)
            row = {
                "timestamp": timestamp,
                "client_id": client_id,
                "activity": activity,
                "features": json.dumps(features)
            }

            if previous_activity is not None and previous_activity != "клик по рекламе"  and previous_activity != "ввод фио" and previous_activity != "сабмит" and previous_activity != "приход в отделение" and random.random() < 0.1:
                data.append({
                    "timestamp": generate_timestamp(start_timestamp, seconds),
                    "client_id": client_id,
                    "activity": previous_activity,
                    "features": json.dumps(generate_features(previous_activity))
                })
                seconds += random.randint(1, 500)
                data.append({
                    "timestamp": generate_timestamp(start_timestamp, seconds),
                    "client_id": client_id,
                    "activity": activity,
                    "features": json.dumps(features)
                })
                seconds += random.randint(1, 500)
            data.append(row)
            previous_activity = activity

    # 10% клиентов делают первые девять активностей
    for client_id in client_ids[90000:92500]:
        start_timestamp = datetime.now() - timedelta(days=random.randint(0, 500))
        seconds = 0
        previous_activity = None
        for activity in activities[:9]:
            timestamp = generate_timestamp(start_timestamp, seconds)
            if activity  == 'клик по рекламе':
              seconds += random.randint(150, 500)
            elif activity  == 'клик на переход с информации на ввод телефона':
              seconds += random.randint(1, 13)
            elif activity  == 'ввод телефона':
              seconds += random.randint(30, 60)
            elif activity  == 'ввод смс':
              seconds += random.randint(20, 60)
            elif activity  == 'ввод паспорта':
              seconds += random.randint(15, 300)
            elif activity  == 'ввод почты':
              seconds += random.randint(5, 45)
            elif activity  == 'ввод фио':
              seconds += random.randint(1, 50)
            elif activity  == 'сабмит':
              seconds += random.randint(230000, 430000)
            else:
              seconds += random.randint(720, 7200)
            features = generate_features(activity)
            row = {
                "timestamp": timestamp,
                "client_id": client_id,
                "activity": activity,
                "features": json.dumps(features)
            }

            if previous_activity is not None and previous_activity != "клик по рекламе"  and previous_activity != "ввод фио" and previous_activity != "сабмит" and previous_activity != "приход в отделение" and random.random() < 0.1:
                data.append({
                    "timestamp": generate_timestamp(start_timestamp, seconds),
                    "client_id": client_id,
                    "activity": previous_activity,
                    "features": json.dumps(generate_features(previous_activity))
                })
                seconds += random.randint(1, 500)
                data.append({
                    "timestamp": generate_timestamp(start_timestamp, seconds),
                    "client_id": client_id,
                    "activity": activity,
                    "features": json.dumps(features)
                })
                seconds += random.randint(1, 500)

            previous_activity = activity
            if activity == "сабмит":
                money = features['value']
            data.append(row)



    # 10% клиентов делают первые десять активностей с повторами
    for client_id in client_ids[92500:100000]:
        start_timestamp = datetime.now() - timedelta(days=random.randint(0, 500))
        seconds = 0
        previous_activity = None
        for activity in activities[:9]:
            timestamp = generate_timestamp(start_timestamp, seconds)
            if activity  == 'клик по рекламе':
              seconds += random.randint(500, 800)
            elif activity  == 'клик на переход с информации на ввод телефона':
              seconds += random.randint(1, 15)
            elif activity  == 'ввод телефона':
              seconds += random.randint(50, 70)
            elif activity  == 'ввод смс':
              seconds += random.randint(40, 70)
            elif activity  == 'ввод паспорта':
              seconds += random.randint(180, 400)
            elif activity  == 'ввод почты':
              seconds += random.randint(15, 60)
            elif activity  == 'ввод фио':
              seconds += random.randint(1, 60)
            elif activity  == 'сабмит':
              seconds += random.randint(80000, 216000)
            else:
              seconds += random.randint(720, 7200)
            features = generate_features(activity, 80, 0, 0, 6, 80, 0, 0, 8, 70, 0, 0, 10, 85, 0, 0, 8)
            row = {
                "timestamp": timestamp,
                "client_id": client_id,
                "activity": activity,
                "features": json.dumps(features)
            }

            if previous_activity is not None and previous_activity != "клик по рекламе"  and previous_activity != "ввод фио" and previous_activity != "сабмит" and previous_activity != "приход в отделение" and random.random() < 0.02:
                data.append({
                    "timestamp": generate_timestamp(start_timestamp, seconds),
                    "client_id": client_id,
                    "activity": previous_activity,
                    "features": json.dumps(generate_features(previous_activity))
                })
                seconds += random.randint(1, 500)
                data.append({
                    "timestamp": generate_timestamp(start_timestamp, seconds),
                    "client_id": client_id,
                    "activity": activity,
                    "features": json.dumps(features)
                })
                seconds += random.randint(1, 500)
            previous_activity = activity
            if activity == "сабмит":
                money = features['value']
            data.append(row)
        if probability_of_taking(money) > random.random():
            timestamp = generate_timestamp(start_timestamp, seconds)
            features = generate_features(activities[9])
            row = {
            "timestamp": timestamp,
            "client_id": client_id,
            "activity": activities[9],
            "features": None
            }
            data.append(row)



    return data

# Создание csv файла
def create_csv_file(file_path):
    data = generate_data()
    fieldnames = ["timestamp", "client_id", "activity", "features"]

    with open(file_path, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)

# Укажите путь к файлу, который нужно создать
file_path = "data.csv"

# Создание csv файла
create_csv_file(file_path)

print(f"CSV файл успешно создан по пути: {file_path}")


CSV файл успешно создан по пути: data.csv
